In [64]:
import numpy as np
from typing import List
import cupy as cp

In [68]:
#参数设置
s0: float = 100
knock_in: float = 90 #敲入水平
knock_out: float = 100 #敲出水平
N: int = 252 #一年的交易日
view_day: List[int] = [i*21-1 for i in range(1,13)] #敲出观察日
T: int = 1 #时间是一年
coupon_rate: float = 0.2 #票息率
r: float = 0 #无风险利率
sigma: float = 0.16 #波动率

In [66]:
def cp_simulate_mc():
    tstep, npath = 252, 1000000
    s0 = 100
    r = 0.05
    sigma = 0.2
    T = 1
    dt = T / tstep
    z = cp.random.normal(size = (tstep + 1, npath))
    st = cp.zeros((tstep + 1, npath))
    st[0] = s0
    for t in range(1, tstep + 1):
        st[t] = st[t - 1] * cp.exp((r - 0.5 * sigma ** 2) * dt + sigma * cp.sqrt(dt) * z[t])
    #先考虑敲出场景

    knock_out_scenario = cp.tile(view_day, (1000000,1)).T  #先生成和观察日有关的网格
    knock_out_scenario = cp.where(st[view_day] > knock_out, knock_out_scenario, cp.inf)  #敲出的时候，记录下观察日
    knock_out_date = np.min(knock_out_scenario, axis=0) #每列最小的数，精确记录每条路径的具体敲出日，如果无敲出则保留inf
    knock_out_month = (knock_out_date + 1) / 21 #每条路径分别第几个月敲出
    is_knock_out = knock_out_month != np.inf
    not_knock_out = knock_out_month == np.inf
    knock_out_year = knock_out_month[is_knock_out] / 12 #把月份化为年
    knock_out_profit = np.sum(knock_out_year * coupon_rate * np.exp(-r * knock_out_year)) #把payoff先折现再求和,计算敲出总所入
    #下面考虑不敲出的情形，分为曾经敲入过和从未敲入过
    knock_in_scenario = np.any(st < knock_in, axis=0) #判断某一条路径是否有敲入
    hold_to_maturity = (knock_in_scenario == False) & not_knock_out #持有到期，没有敲入也没有敲出
    hold_to_maturity_count = np.count_nonzero(hold_to_maturity) #平稳持有到期路径条数
    htm_profit = hold_to_maturity_count * coupon_rate * np.exp(-r * T) #平稳持有到期收入
    loss = np.sum((st[-1, not_knock_out & (knock_in_scenario == True) & (st[-1] < s0)] / s0 - 1) * np.exp(-r * T)) #敲入造成的总亏损，对于st>s0的情况，损益为0，不需要考虑
    price = (htm_profit + knock_out_profit + loss)/1000000
    return price

In [67]:
%timeit cp_simulate_mc()

349 ms ± 171 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [68]:
def st_simulate_mc():
    tstep, npath = 252, 1000000
    s0 = 100
    r = 0.05
    sigma = 0.2
    T = 1
    dt = T / tstep
    z = cp.random.normal(size = (tstep + 1, npath))
    path = cp.zeros((tstep + 1, npath))
    path[0] = s0
    for t in range(1, tstep + 1):
        path[t] = path[t - 1] * cp.exp((r - 0.5 * sigma ** 2) * dt + sigma * cp.sqrt(dt) * z[t])
    return path

In [69]:
%timeit st_simulate_mc()

KeyboardInterrupt: 

In [69]:
epsilon = np.random.standard_normal((251,300000)) #正态分布30万条路径

In [70]:
dt = T / N #年化时间
log_r = (r - 0.5 * sigma ** 2) * dt + sigma * np.sqrt(dt) * epsilon #log(ST/St)
log_r = np.concatenate([np.zeros((1, 300000)), log_r], axis = 0) #第一天0收益

In [71]:
st = np.cumprod(np.exp(log_r), axis = 0) * s0 #30万条标的资产路径

In [72]:
#先考虑敲出场景

knock_out_scenario = np.tile(view_day, (300000,1)).T  #先生成和观察日有关的网格
knock_out_scenario = np.where(st[view_day] > knock_out, knock_out_scenario, np.inf)  #敲出的时候，记录下观察日

In [73]:
knock_out_scenario

array([[ 20.,  inf,  20., ...,  20.,  inf,  20.],
       [ 41.,  41.,  41., ...,  41.,  inf,  41.],
       [ 62.,  62.,  62., ...,  inf,  inf,  62.],
       ...,
       [209., 209.,  inf, ..., 209.,  inf, 209.],
       [230.,  inf,  inf, ..., 230.,  inf, 230.],
       [251., 251.,  inf, ..., 251.,  inf, 251.]])

In [74]:
knock_out_date = np.min(knock_out_scenario, axis=0) #每列最小的数，精确记录每条路径的具体敲出日，如果无敲出则保留inf

In [75]:
knock_out_date

array([20., 41., 20., ..., 20., inf, 20.])

In [76]:
knock_out_month = (knock_out_date + 1) / 21 #每条路径分别第几个月敲出

In [77]:
knock_out_month

array([ 1.,  2.,  1., ...,  1., inf,  1.])

In [78]:
is_knock_out = knock_out_month != np.inf
not_knock_out = knock_out_month == np.inf

In [79]:
is_knock_out

array([ True,  True,  True, ...,  True, False,  True])

In [80]:
knock_out_year = knock_out_month[is_knock_out] / 12 #把月份化为年

In [81]:
len(knock_out_year)

247975

In [82]:
knock_out_year * coupon_rate #计算敲出路径的payoff

array([0.01666667, 0.03333333, 0.01666667, ..., 0.01666667, 0.01666667,
       0.01666667])

In [83]:
knock_out_profit = np.sum(knock_out_year * coupon_rate * np.exp(-r * knock_out_year)) #把payoff先折现再求和,计算敲出总所入


In [84]:
knock_out_profit

9516.75

In [85]:
#下面考虑不敲出的情形，分为曾经敲入过和从未敲入过

knock_in_scenario = np.any(st < knock_in, axis=0) #判断某一条路径是否有敲入


300000

In [86]:
hold_to_maturity = (knock_in_scenario == False) & not_knock_out #持有到期，没有敲入也没有敲出


In [87]:
hold_to_maturity_count = np.count_nonzero(hold_to_maturity) #平稳持有到期路径条数


In [88]:
hold_to_maturity_count

380

In [89]:
htm_profit = hold_to_maturity_count * coupon_rate * np.exp(-r * T) #平稳持有到期收入


In [90]:
htm_profit #平稳持有到期总收入折现

76.0

In [91]:
loss = np.sum((st[-1, not_knock_out & (knock_in_scenario == True) & (st[-1] < s0)] / s0 - 1) * np.exp(-r * T)) #敲入造成的总亏损，对于st>s0的情况，损益为0，不需要考虑


In [92]:
loss

-8337.136302746714

In [67]:
(htm_profit + knock_out_profit + loss)/300000

0.0038437581781596477

In [23]:
#（附注：陈帅先当时在南华做的结果是0.05087，与我很接近）

In [60]:
%timeit np.linalg.inv(np.array([[1, 2],[3, 4]])) #使用cpu求逆矩阵

12.3 µs ± 376 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [61]:
%timeit cp.linalg.inv(cp.array([[1, 2],[3, 4]])) # GPU求逆矩阵

170 µs ± 78.2 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [1]:
import numpy as np
from typing import List
import cupy as cp

In [1363]:
s0: float = 100
knock_in: float = 110 #敲入水平
knock_out: float = 100 #敲出水平
N: int = 252 #一年的交易日
view_day: List[int] = [i*21-1 for i in range(1,25)] #敲出观察日
print(view_day)
T: int = 2 #时间是一年
coupon_rate: float = 0.0455 #票息率
r: float = 0 #无风险利率
sigma: float = 0.14 #波动率
q: float = 0.05 #分红率
drift: float = r - q #漂移率
print(drift)

[20, 41, 62, 83, 104, 125, 146, 167, 188, 209, 230, 251, 272, 293, 314, 335, 356, 377, 398, 419, 440, 461, 482, 503]
-0.05


In [1364]:

    tstep, npath = 504, 500000
    dt = T / (tstep - 1)
    z = cp.random.normal(size = (tstep + 1, npath))
    st = cp.zeros((tstep + 1, npath))
    st[0] = s0
    for t in range(1, tstep + 1):
        st[t] = st[t - 1] * cp.exp((drift - 0.5 * sigma ** 2) * dt + sigma * cp.sqrt(dt) * z[t])
    #先考虑敲出场景

    knock_out_scenario = cp.tile(view_day, (npath,1)).T  #先生成和观察日有关的网格
    knock_out_scenario = cp.where(st[view_day] < knock_out, knock_out_scenario, cp.inf)  #向下敲出的时候，记录下观察日
    knock_out_date = np.min(knock_out_scenario, axis=0) #每列最小的数，精确记录每条路径的具体敲出日，如果无敲出则保留inf
    knock_out_month = (knock_out_date + 1) / 21 #每条路径分别第几个月敲出
    is_knock_out = knock_out_month != np.inf
    not_knock_out = knock_out_month == np.inf
    knock_out_year = knock_out_month[is_knock_out] / 12 #把月份化为年
    print(len(knock_out_year))
    knock_out_profit = np.sum(knock_out_year * s0 * coupon_rate * np.exp(-r * knock_out_year)) #把payoff先折现再求和,计算敲出总所入
knock_out_profit

474716


array(527850.42916667)

In [1365]:
    #下面考虑不敲出的情形，分为曾经敲入过和从未敲入过
    knock_in_scenario = np.any(st > knock_in, axis=0) #判断某一条路径是否有向上敲入
    hold_to_maturity = (knock_in_scenario == False) & not_knock_out #持有到期，没有敲入也没有敲出
    hold_to_maturity_count = np.count_nonzero(hold_to_maturity) #平稳持有到期路径条数
    htm_profit = hold_to_maturity_count * s0 * coupon_rate * np.exp(-r * T) #平稳持有到期收入
htm_profit

array(9.1)

In [1366]:
    loss = np.sum((1 - st[-1, not_knock_out & (knock_in_scenario == True) & (st[-1] > s0)] / s0) * s0 * np.exp(-r * T)) #敲入造成的总亏损，对于st>s0的情况，损益为0，不需要考虑
    loss

array(-525064.56772814)

In [1367]:
price = (htm_profit + knock_out_profit + loss)/npath
price

array(0.00558992)

In [768]:
#参数设置
s0: float = 100
knock_in: float = 90 #敲入水平
knock_out: float = 100#敲出水平
N: int = 252 #一年的交易日
view_day: List[int] = [i*21-1 for i in range(1,13)] #敲出观察日
T: int = 1 #时间是一年
coupon_rate: float = 0.2 #票息率
r: float = 0 #无风险利率
sigma: float = 0.16 #波动率
q: float = 0 #分红率
drift: float = r - q #漂移率

In [123]:
    tstep, npath = 252, 300000
    dt = T / tstep
    z = cp.random.normal(size = (tstep + 1, npath))
    st = cp.zeros((tstep + 1, npath))
    st[0] = s0
    for t in range(1, tstep + 1):
        st[t] = st[t - 1] * cp.exp((drift - 0.5 * sigma ** 2) * dt + sigma * cp.sqrt(dt) * z[t])
    #先考虑敲出场景

    knock_out_scenario = cp.tile(view_day, (300000,1)).T  #先生成和观察日有关的网格
    knock_out_scenario = cp.where(st[view_day] > knock_out, knock_out_scenario, cp.inf)  #向上敲出的时候，记录下观察日
    knock_out_date = np.min(knock_out_scenario, axis=0) #每列最小的数，精确记录每条路径的具体敲出日，如果无敲出则保留inf
    knock_out_month = (knock_out_date + 1) / 21 #每条路径分别第几个月敲出
    is_knock_out = knock_out_month != np.inf
    not_knock_out = knock_out_month == np.inf
    knock_out_year = knock_out_month[is_knock_out] / 12 #把月份化为年
    print(len(knock_out_year))
    knock_out_profit = np.sum(knock_out_year * coupon_rate * np.exp(-r * knock_out_year)) #把payoff先折现再求和,计算敲出总所入
knock_out_profit

248303


array(9516.6)

In [101]:
    #下面考虑不敲出的情形，分为曾经敲入过和从未敲入过
    knock_in_scenario = np.any(st < knock_in, axis=0) #判断某一条路径是否有向下敲入
    hold_to_maturity = (knock_in_scenario == False) & not_knock_out #持有到期，没有敲入也没有敲出
    hold_to_maturity_count = np.count_nonzero(hold_to_maturity) #平稳持有到期路径条数
    htm_profit = hold_to_maturity_count * coupon_rate * np.exp(-r * T) #平稳持有到期收入
htm_profit

array(72.)

In [103]:
    loss = np.sum((st[-1, not_knock_out & (knock_in_scenario == True) & (st[-1] > s0)] / s0 - 1) * np.exp(-r * T)) #敲入造成的总亏损，对于st>s0的情况，损益为0，不需要考虑
    loss

array(1.73569841)

In [104]:
price = (htm_profit + knock_out_profit + loss)/300000
price

array(0.03194445)